# Making a better USG% metric

USG is currently:

Definition:
- The percentage of team plays used by a player when they are on the floor 

Formula:
- (FGA + Possession Ending FTA + TO) / POSS 


I would like to make an offensive_load metric that is more of an overall offensive_load rating. I really don't think turnovers should be included in the USG metric, at least for what I use USG for. So maybe, not a BETTER metric, just a different one, more specific to my needs.

The offensive_load metric would be:
- (FGA / TEAM FGA) + (FTA / TEAM FTA)

The offensive_load per minute metric would be:
- ((FGA / TEAM FGA) + (FTA / TEAM FTA)) / MIN

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil       
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
import xgboost
from xgboost import XGBClassifier
from random import randint
import  random
import os
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import make_column_selector as selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
import pickle
from sklearn.metrics import fbeta_score
import winsound
from sklearn.linear_model import LinearRegression
from sklearn import tree, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, f1_score, make_scorer, recall_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import streamlit as st

today = datetime.datetime.today().strftime('%Y-%m-%d')

In [59]:
# load updated player data
trad_player_data = pd.read_csv('data/player/aggregates/Trad&Adv_box_scores_GameView.csv')
# fix date so it is sortable
trad_player_data['trad_game date'] = pd.to_datetime(trad_player_data['trad_game date'])
# convert to just date
trad_player_data['trad_game date'] = trad_player_data['trad_game date'].dt.date
# to date
#trad_player_data['trad_game date'] = trad_player_data['trad_game date'].dt.strftime('%Y-%m-%d')
# sort by date
trad_player_data = trad_player_data.sort_values(by = ['trad_game date'], ascending = False)
# drop nan for trad_player
trad_player_data = trad_player_data.dropna(subset = ['trad_player'])
trad_player_data.head()

,Unnamed: 0,trad_unnamed: 0,trad_player,trad_team,trad_match up,trad_game date,trad_w/l,trad_min,trad_pts,trad_fgm,...,adv_dreb%,adv_reb%,adv_to ratio,adv_efg%,adv_ts%,adv_usg%,adv_pace,adv_pie,adv_season,adv_season_type
263003,263483,172,Nickeil Alexander-Walker,UTA,UTA @ MIN,2023-01-16,W,10.0,5.0,2.0,...,10.0,9.1,37.5,83.3,83.3,30.0,94.27,6.3,2022,Regular
262971,263451,140,Bogdan Bogdanovic,ATL,ATL vs. MIA,2023-01-16,W,31.0,8.0,2.0,...,3.1,1.8,0.0,27.3,33.7,18.2,98.73,4.8,2022,Regular
262981,263461,150,Kristaps Porzingis,WAS,WAS vs. GSW,2023-01-16,L,37.0,32.0,9.0,...,13.5,7.1,4.2,70.0,78.9,25.0,104.77,17.8,2022,Regular
262980,263460,149,Jordan Poole,GSW,GSW @ WAS,2023-01-16,W,37.0,32.0,12.0,...,21.2,10.3,14.8,77.5,78.3,27.6,103.39,17.8,2022,Regular
262979,263459,148,Bam Adebayo,MIA,MIA @ ATL,2023-01-16,L,36.0,20.0,9.0,...,25.0,19.7,10.7,47.4,50.3,25.3,100.87,13.8,2022,Regular


In [60]:
# add team totals for each game for points, rebounds, assists, turnovers, steals, free throws attempted, and shots attempted
trad_player_data = trad_player_data.merge(trad_player_data.groupby(['trad_game date', 'trad_team'])['trad_pts', 'trad_reb', 'trad_ast', 'trad_tov', 'trad_stl', 'trad_fta', 'trad_fga'].sum().reset_index().rename(columns={'trad_pts': 'trad_team_pts', 'trad_reb': 'trad_team_reb', 'trad_ast': 'trad_team_ast', 'trad_tov': 'trad_team_tov', 'trad_stl': 'trad_team_stl', 'trad_fta': 'trad_team_fta', 'trad_fga': 'trad_team_fga'}), how='left', left_on=['trad_game date', 'trad_team'], right_on=['trad_game date', 'trad_team'])

trad_player_data.head()

,Unnamed: 0,trad_unnamed: 0,trad_player,trad_team,trad_match up,trad_game date,trad_w/l,trad_min,trad_pts,trad_fgm,...,adv_pie,adv_season,adv_season_type,trad_team_pts,trad_team_reb,trad_team_ast,trad_team_tov,trad_team_stl,trad_team_fta,trad_team_fga
0,263483,172,Nickeil Alexander-Walker,UTA,UTA @ MIN,2023-01-16,W,10.0,5.0,2.0,...,6.3,2022,Regular,126.0,50.0,25.0,18.0,0.0,28.0,84.0
1,263451,140,Bogdan Bogdanovic,ATL,ATL vs. MIA,2023-01-16,W,31.0,8.0,2.0,...,4.8,2022,Regular,121.0,28.0,27.0,16.0,9.0,22.0,76.0
2,263461,150,Kristaps Porzingis,WAS,WAS vs. GSW,2023-01-16,L,37.0,32.0,9.0,...,17.8,2022,Regular,118.0,36.0,31.0,14.0,7.0,23.0,86.0
3,263460,149,Jordan Poole,GSW,GSW @ WAS,2023-01-16,W,37.0,32.0,12.0,...,17.8,2022,Regular,127.0,43.0,28.0,12.0,8.0,21.0,91.0
4,263459,148,Bam Adebayo,MIA,MIA @ ATL,2023-01-16,L,36.0,20.0,9.0,...,13.8,2022,Regular,113.0,41.0,30.0,14.0,10.0,20.0,89.0


In [61]:
# add player_points_percentage, player_rebounds_percentage, player_assists_percentage, player_turnovers_percentage, player_steals_percentage, player_free_throws_percentage, and player_field_goal_percentage
trad_player_data['player_points_percentage'] = trad_player_data['trad_pts'] / trad_player_data['trad_team_pts']
trad_player_data['player_rebounds_percentage'] = trad_player_data['trad_reb'] / trad_player_data['trad_team_reb']
trad_player_data['player_assists_percentage'] = trad_player_data['trad_ast'] / trad_player_data['trad_team_ast']
trad_player_data['player_turnovers_percentage'] = trad_player_data['trad_tov'] / trad_player_data['trad_team_tov']
trad_player_data['player_steals_percentage'] = trad_player_data['trad_stl'] / trad_player_data['trad_team_stl']
trad_player_data['player_free_throws_attempted_percentage'] = trad_player_data['trad_fta'] / trad_player_data['trad_team_fta']
trad_player_data['player_field_goal_attepted_percentage'] = trad_player_data['trad_fga'] / trad_player_data['trad_team_fga']


In [62]:
# add per_min for each of the above
trad_player_data['player_points_percentage_per_min'] = trad_player_data['player_points_percentage'] / trad_player_data['trad_min'] * 100
trad_player_data['player_rebounds_percentage_per_min'] = trad_player_data['player_rebounds_percentage'] / trad_player_data['trad_min'] * 100
trad_player_data['player_assists_percentage_per_min'] = trad_player_data['player_assists_percentage'] / trad_player_data['trad_min'] * 100
trad_player_data['player_turnovers_percentage_per_min'] = trad_player_data['player_turnovers_percentage'] / trad_player_data['trad_min'] * 100
trad_player_data['player_steals_percentage_per_min'] = trad_player_data['player_steals_percentage'] / trad_player_data['trad_min'] * 100




In [63]:
# fix nans with 0
trad_player_data = trad_player_data.fillna(0)
trad_player_data

,Unnamed: 0,trad_unnamed: 0,trad_player,trad_team,trad_match up,trad_game date,trad_w/l,trad_min,trad_pts,trad_fgm,...,player_assists_percentage,player_turnovers_percentage,player_steals_percentage,player_free_throws_attempted_percentage,player_field_goal_attepted_percentage,player_points_percentage_per_min,player_rebounds_percentage_per_min,player_assists_percentage_per_min,player_turnovers_percentage_per_min,player_steals_percentage_per_min
0,263483,172,Nickeil Alexander-Walker,UTA,UTA @ MIN,2023-01-16,W,10.0,5.0,2.0,...,0.080000,0.166667,0.000000,0.000000,0.035714,0.396825,0.200000,0.800000,1.666667,0.000000
1,263451,140,Bogdan Bogdanovic,ATL,ATL vs. MIA,2023-01-16,W,31.0,8.0,2.0,...,0.185185,0.000000,0.222222,0.090909,0.144737,0.213276,0.115207,0.597372,0.000000,0.716846
2,263461,150,Kristaps Porzingis,WAS,WAS vs. GSW,2023-01-16,L,37.0,32.0,9.0,...,0.096774,0.071429,0.142857,0.521739,0.174419,0.732936,0.375375,0.261552,0.193050,0.386100
3,263460,149,Jordan Poole,GSW,GSW @ WAS,2023-01-16,W,37.0,32.0,12.0,...,0.107143,0.333333,0.250000,0.047619,0.219780,0.680996,0.439975,0.289575,0.900901,0.675676
4,263459,148,Bam Adebayo,MIA,MIA @ ATL,2023-01-16,L,36.0,20.0,9.0,...,0.166667,0.214286,0.300000,0.100000,0.213483,0.491642,0.880759,0.462963,0.595238,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265291,221,215,Jared Sullinger,BOS,BOS @ MIA,2012-10-30,L,8.0,2.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.026667,0.233645,0.304878,0.000000,0.000000,0.000000
265292,222,216,Udonis Haslem,MIA,MIA vs. BOS,2012-10-30,W,11.0,0.0,0.0,...,0.000000,0.125000,0.000000,0.000000,0.012658,0.000000,0.757576,0.000000,1.136364,0.000000
265293,223,217,Mike Miller,MIA,MIA vs. BOS,2012-10-30,W,7.0,0.0,0.0,...,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.571429,0.000000,0.000000
265294,224,218,LeBron James,MIA,MIA vs. BOS,2012-10-30,W,29.0,26.0,10.0,...,0.120000,0.000000,0.250000,0.156250,0.202532,0.747126,0.957854,0.413793,0.000000,0.862069


In [64]:
# get this season averages
tpd22 = trad_player_data[trad_player_data['trad_game date'] >= datetime.date(2022, 10, 1)]
# get averages
tpd22_avg = tpd22.groupby(['trad_player'])['trad_pts', 'trad_reb', 'trad_ast', 'trad_tov', 'trad_stl', 'trad_fta', 'trad_fga', 'player_points_percentage', 'player_rebounds_percentage', 'player_assists_percentage', 'player_turnovers_percentage', 'player_steals_percentage', 'player_free_throws_attempted_percentage', 'player_field_goal_attepted_percentage', 'player_points_percentage_per_min', 'player_rebounds_percentage_per_min', 'player_assists_percentage_per_min', 'player_turnovers_percentage_per_min', 'player_steals_percentage_per_min'].mean().reset_index()
tpd22_avg

,trad_player,trad_pts,trad_reb,trad_ast,trad_tov,trad_stl,trad_fta,trad_fga,player_points_percentage,player_rebounds_percentage,player_assists_percentage,player_turnovers_percentage,player_steals_percentage,player_free_throws_attempted_percentage,player_field_goal_attepted_percentage,player_points_percentage_per_min,player_rebounds_percentage_per_min,player_assists_percentage_per_min,player_turnovers_percentage_per_min,player_steals_percentage_per_min
0,A.J. Lawson,1.375000,0.500000,0.125000,0.000000,0.000000,0.125000,1.250000,0.011233,0.015236,0.004464,0.000000,0.000000,0.003788,0.014802,0.266176,0.682406,0.063776,0.000000,0.000000
1,AJ Green,5.047619,1.142857,0.428571,0.190476,0.190476,0.190476,3.761905,0.047378,0.024065,0.016863,0.014503,0.031049,0.008287,0.042345,0.364462,0.178801,0.272812,0.113746,0.221943
2,AJ Griffin,9.900000,2.200000,1.000000,0.725000,0.825000,0.400000,8.250000,0.085941,0.051196,0.043206,0.064039,0.112464,0.018185,0.090046,0.424951,0.256112,0.201096,0.294535,0.531738
3,Aaron Gordon,16.736842,6.842105,2.447368,1.605263,0.842105,4.789474,10.736842,0.142595,0.156421,0.083181,0.109158,0.114338,0.206004,0.124329,0.467990,0.519148,0.270113,0.379707,0.380671
4,Aaron Holiday,4.500000,1.425000,1.475000,0.600000,0.625000,0.650000,4.025000,0.039198,0.033293,0.064589,0.060279,0.080967,0.030918,0.043382,0.220948,0.197851,0.358749,0.328000,0.430908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,Zach Collins,9.218750,5.781250,2.468750,1.937500,0.375000,2.156250,6.750000,0.079967,0.132257,0.090337,0.129728,0.059736,0.099070,0.073351,0.386403,0.647795,0.448534,0.641740,0.298997
491,Zach LaVine,24.153846,4.589744,4.358974,2.435897,1.025641,4.948718,18.230769,0.210945,0.111508,0.179084,0.184596,0.146827,0.229485,0.208734,0.588179,0.312523,0.501953,0.518106,0.414743
492,Zeke Nnaji,4.727273,1.818182,0.242424,0.454545,0.181818,0.909091,3.151515,0.040109,0.042028,0.009106,0.036399,0.041667,0.039179,0.036604,0.305417,0.457962,0.109744,0.594921,0.802196
493,Ziaire Williams,6.588235,2.941176,1.000000,1.235294,0.470588,0.588235,6.000000,0.053089,0.059603,0.036732,0.092351,0.047114,0.022663,0.064498,0.314274,0.376194,0.190029,0.670036,0.219436


In [65]:
# rank by player_points_percentage_per_min
tpd22_avg = tpd22_avg.sort_values(by = ['player_field_goal_attepted_percentage'], ascending = False).round(2)
tpd22_avg.head(50)

,trad_player,trad_pts,trad_reb,trad_ast,trad_tov,trad_stl,trad_fta,trad_fga,player_points_percentage,player_rebounds_percentage,player_assists_percentage,player_turnovers_percentage,player_steals_percentage,player_free_throws_attempted_percentage,player_field_goal_attepted_percentage,player_points_percentage_per_min,player_rebounds_percentage_per_min,player_assists_percentage_per_min,player_turnovers_percentage_per_min,player_steals_percentage_per_min
312,Luka Doncic,33.33,9.03,8.82,3.74,1.54,11.21,22.64,0.30,0.23,0.40,0.29,0.22,0.42,0.27,0.79,0.61,1.08,0.79,0.58
231,Joel Embiid,33.97,9.71,4.10,3.65,1.06,11.81,21.00,0.30,0.25,0.17,0.27,0.13,0.46,0.25,0.85,0.71,0.49,0.78,0.38
307,LeBron James,29.71,8.41,6.97,3.18,1.00,6.09,22.62,0.25,0.19,0.27,0.22,0.15,0.25,0.25,0.70,0.52,0.74,0.63,0.42
219,Jayson Tatum,31.09,8.26,4.28,2.60,1.02,8.56,21.60,0.26,0.19,0.16,0.21,0.14,0.38,0.24,0.71,0.51,0.43,0.56,0.40
124,Donovan Mitchell,28.72,3.90,4.72,2.87,1.36,5.92,20.26,0.25,0.09,0.20,0.22,0.21,0.24,0.24,0.69,0.25,0.54,0.61,0.57
301,Kyrie Irving,25.55,4.81,4.84,2.26,0.77,3.87,19.97,0.22,0.12,0.19,0.16,0.11,0.20,0.24,0.61,0.33,0.52,0.44,0.30
86,Damian Lillard,28.84,3.90,7.06,3.19,0.87,8.06,19.71,0.25,0.09,0.30,0.21,0.13,0.30,0.23,0.70,0.25,0.83,0.59,0.35
154,Giannis Antetokounmpo,31.00,11.94,5.34,4.00,0.80,12.94,20.83,0.27,0.24,0.22,0.28,0.12,0.53,0.23,0.80,0.70,0.64,0.82,0.35
216,Jaylen Brown,27.15,7.08,3.20,3.05,1.08,5.48,20.42,0.23,0.16,0.12,0.23,0.16,0.24,0.23,0.66,0.44,0.34,0.64,0.47
116,Devin Booker,27.10,4.62,5.55,2.41,0.90,6.21,20.45,0.23,0.11,0.21,0.19,0.12,0.31,0.23,0.66,0.30,0.58,0.53,0.32


In [66]:
# keep just new columns
current_usg_metrics = tpd22_avg[['trad_player', 'player_points_percentage', 'player_rebounds_percentage', 
                                'player_assists_percentage', 'player_turnovers_percentage', 'player_steals_percentage', 
                                'player_free_throws_attempted_percentage', 'player_field_goal_attepted_percentage', 
                                'player_points_percentage_per_min', 'player_rebounds_percentage_per_min', 'player_assists_percentage_per_min', 
                                'player_turnovers_percentage_per_min', 'player_steals_percentage_per_min']]

In [67]:
current_usg_metrics

,trad_player,player_points_percentage,player_rebounds_percentage,player_assists_percentage,player_turnovers_percentage,player_steals_percentage,player_free_throws_attempted_percentage,player_field_goal_attepted_percentage,player_points_percentage_per_min,player_rebounds_percentage_per_min,player_assists_percentage_per_min,player_turnovers_percentage_per_min,player_steals_percentage_per_min
312,Luka Doncic,0.30,0.23,0.40,0.29,0.22,0.42,0.27,0.79,0.61,1.08,0.79,0.58
231,Joel Embiid,0.30,0.25,0.17,0.27,0.13,0.46,0.25,0.85,0.71,0.49,0.78,0.38
307,LeBron James,0.25,0.19,0.27,0.22,0.15,0.25,0.25,0.70,0.52,0.74,0.63,0.42
219,Jayson Tatum,0.26,0.19,0.16,0.21,0.14,0.38,0.24,0.71,0.51,0.43,0.56,0.40
124,Donovan Mitchell,0.25,0.09,0.20,0.22,0.21,0.24,0.24,0.69,0.25,0.54,0.61,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Braxton Key,0.01,0.01,0.00,0.00,0.00,0.02,0.00,0.25,0.48,0.00,0.00,0.00
319,Malcolm Hill,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,inf,0.00,0.00,0.00
14,Alondes Williams,0.00,0.02,0.00,0.11,0.00,0.00,0.00,0.00,0.34,0.00,2.22,0.00
341,Michael Foster Jr.,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [68]:
# add percentiles for each metric
current_usg_metrics['player_points_percentage_percentile'] = current_usg_metrics['player_points_percentage'].rank(pct=True)
current_usg_metrics['player_rebounds_percentage_percentile'] = current_usg_metrics['player_rebounds_percentage'].rank(pct=True)
current_usg_metrics['player_assists_percentage_percentile'] = current_usg_metrics['player_assists_percentage'].rank(pct=True)
current_usg_metrics['player_turnovers_percentage_percentile'] = current_usg_metrics['player_turnovers_percentage'].rank(pct=True)
current_usg_metrics['player_steals_percentage_percentile'] = current_usg_metrics['player_steals_percentage'].rank(pct=True)
current_usg_metrics['player_free_throws_attempted_percentage_percentile'] = current_usg_metrics['player_free_throws_attempted_percentage'].rank(pct=True)
current_usg_metrics['player_field_goal_attepted_percentage_percentile'] = current_usg_metrics['player_field_goal_attepted_percentage'].rank(pct=True)
current_usg_metrics['player_points_percentage_per_min_percentile'] = current_usg_metrics['player_points_percentage_per_min'].rank(pct=True)
current_usg_metrics['player_rebounds_percentage_per_min_percentile'] = current_usg_metrics['player_rebounds_percentage_per_min'].rank(pct=True)
current_usg_metrics['player_assists_percentage_per_min_percentile'] = current_usg_metrics['player_assists_percentage_per_min'].rank(pct=True)
current_usg_metrics['player_turnovers_percentage_per_min_percentile'] = current_usg_metrics['player_turnovers_percentage_per_min'].rank(pct=True)
current_usg_metrics['player_steals_percentage_per_min_percentile'] = current_usg_metrics['player_steals_percentage_per_min'].rank(pct=True)

In [69]:
current_usg_metrics = current_usg_metrics.sort_values(by = ['player_points_percentage_percentile'], ascending = False).round(2)
current_usg_metrics.head(10)

,trad_player,player_points_percentage,player_rebounds_percentage,player_assists_percentage,player_turnovers_percentage,player_steals_percentage,player_free_throws_attempted_percentage,player_field_goal_attepted_percentage,player_points_percentage_per_min,player_rebounds_percentage_per_min,...,player_assists_percentage_percentile,player_turnovers_percentage_percentile,player_steals_percentage_percentile,player_free_throws_attempted_percentage_percentile,player_field_goal_attepted_percentage_percentile,player_points_percentage_per_min_percentile,player_rebounds_percentage_per_min_percentile,player_assists_percentage_per_min_percentile,player_turnovers_percentage_per_min_percentile,player_steals_percentage_per_min_percentile
312,Luka Doncic,0.30,0.23,0.40,0.29,0.22,0.42,0.27,0.79,0.61,...,1.00,1.00,0.98,0.99,1.00,1.00,0.83,0.99,0.97,0.83
231,Joel Embiid,0.30,0.25,0.17,0.27,0.13,0.46,0.25,0.85,0.71,...,0.87,0.99,0.82,1.00,1.00,1.00,0.90,0.75,0.96,0.51
154,Giannis Antetokounmpo,0.27,0.24,0.22,0.28,0.12,0.53,0.23,0.80,0.70,...,0.93,1.00,0.77,1.00,0.98,1.00,0.89,0.87,0.97,0.44
425,Shai Gilgeous-Alexander,0.27,0.11,0.23,0.23,0.22,0.44,0.22,0.75,0.31,...,0.94,0.97,0.98,1.00,0.97,0.99,0.37,0.87,0.91,0.86
219,Jayson Tatum,0.26,0.19,0.16,0.21,0.14,0.38,0.24,0.71,0.51,...,0.85,0.94,0.86,0.99,0.99,0.98,0.74,0.68,0.81,0.55
285,Kevin Durant,0.26,0.17,0.20,0.26,0.11,0.36,0.23,0.73,0.47,...,0.91,0.99,0.72,0.99,0.98,0.99,0.70,0.82,0.93,0.33
307,LeBron James,0.25,0.19,0.27,0.22,0.15,0.25,0.25,0.70,0.52,...,0.97,0.95,0.89,0.95,1.00,0.98,0.75,0.91,0.88,0.61
124,Donovan Mitchell,0.25,0.09,0.20,0.22,0.21,0.24,0.24,0.69,0.25,...,0.91,0.95,0.97,0.95,0.99,0.98,0.20,0.80,0.86,0.82
86,Damian Lillard,0.25,0.09,0.30,0.21,0.13,0.30,0.23,0.70,0.25,...,0.98,0.94,0.82,0.97,0.98,0.98,0.20,0.95,0.84,0.44
430,Stephen Curry,0.25,0.15,0.21,0.19,0.13,0.27,0.22,0.72,0.43,...,0.92,0.92,0.82,0.96,0.97,0.99,0.64,0.85,0.81,0.51


In [70]:
# save as current usg metrics
current_usg_metrics.to_csv('data/player/aggregates/current_usg_metrics_'+ today +'.csv', index = False)